### 1.0 Introduction

The objective of the project is to get information about heatwave warning systems for various countries that can complement/Complete the World Meteorological Organization (WMO) heat wave warning systems inventory.

In this section of the project we scrap data from google scholar and from a google search using the Key words ("heatwave", "heat wave","heat  wave","heat  action  plan","heat  health","heat early warning system","heat-health action plan",
"heat-health warning  systems", "extreme heat management","heat wave management", "public health heat wave alert"). We will attempt to scrap for URLs, Authors, Publication dates, Journal Titles, abstracts, citations, web page content and countries.

### 2.0 Data scrapping

#### 2.1 Scrapping data from google scholar

In [1]:
import requests
from bs4 import BeautifulSoup
import csv

def scrape_google_scholar(keywords):
    """Scrapes the Google Scholar website for papers and journals that contain the given keywords.

    Args:
        keywords: A list of keywords to search for.

    Returns:
        A list of dictionaries, where each dictionary contains information about a
        paper or journal, including author(s), title, publication date, source, page numbers, and abstract.
    """

    # Combine the keywords into a single search query.
    query = "+".join(keywords)

    # Make a request to the Google Scholar website.
    response = requests.get("https://scholar.google.com/scholar?hl=en&q=" + query)

    # Parse the HTML response.
    soup = BeautifulSoup(response.content, "html.parser")

    # Extract the data from the HTML.
    data = []

    # Find all of the search results.
    search_results = soup.find_all("div", class_="gs_r")

    # Iterate over the search results and extract the data.
    for search_result in search_results:
        try:
            author = search_result.find("div", class_="gs_a").text
        except AttributeError:
            author = "None"

        try:
            title_link = search_result.find("h3", class_="gs_rt").find("a")
            title = title_link.text
            # Get the URL of the paper from the link.
            link = title_link.get("href")
        except AttributeError:
            title = "None"
            link = "None"

        try:
            source = search_result.find("div", class_="gs_a").text
        except AttributeError:
            source = "None"

        try:
            publication_date = search_result.find("div", class_="gs_a").text
        except AttributeError:
            publication_date = "None"

        try:
            # Extract page numbers if available.
            page_numbers = search_result.find("div", class_="gs_fl").text
        except AttributeError:
            page_numbers = "None"

        try:
            # Try to locate the abstract within the search result.
            abstract_element = search_result.find("div", class_="gs_rs")
            if abstract_element:
                abstract = abstract_element.text
            else:
                abstract = "None"
        except AttributeError:
            abstract = "None"

        # Create a dictionary to store the scraped data.
        data.append({
            "Author(s)": author,
            "Title": title,
            "Publication Date": publication_date,
            "Source": source,
            "Page Numbers": page_numbers,
            "Link": link,
            "Abstract": abstract
        })

    return data

# List of keywords to search for.
keywords = ["heatwave", "heat wave","heat  wave","heat  action  plan","heat  health","heat early warning system","heat-health action plan",
            "heat-health warning  systems", "extreme heat management","heat wave management", "public health heat wave alert"]

# Scrape Google Scholar and save the data to a CSV file.
with open("google_scholar_results_with_abstracts_fixed_2.csv", "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["Author(s)", "Title", "Publication Date", "Source", "Page Numbers", "Link", "Abstract"])
    writer.writeheader()

    for keyword in keywords:
        for data in scrape_google_scholar([keyword]):
            writer.writerow(data)


#### 2.2 Perform a Google Search using the key words and extract URLs

Here we just extract urls from the search result. After that we will  classify the URLs into categories i.e Journals, government websites, news media, e.t.c

##### a. Perform google search and extract URLs

In [3]:
import requests
import re
import csv

def search_google(keywords):
    """Search Google for the specified keywords and collect URLs.

    Args:
        keywords: A list of keywords to search for.

    Returns:
        A list of URLs containing any of the specified keywords.
    """
    search_results = []

    # Perform a Google search for each keyword.
    for keyword in keywords:
        query = f"https://www.google.com/search?q={keyword.replace(' ', '+')}"
        response = requests.get(query)
        
        if response.status_code == 200:
            # Extract URLs from the search results using regular expressions.
            urls = re.findall(r"https?://[^\s/$.?#].[^\s]*", response.text)
            
            # Filter URLs to exclude Google search result pages.
            valid_urls = [url for url in urls if "google.com" not in url]
            
            # Add the valid URLs to the list.
            search_results.extend(valid_urls)

    return search_results

# List of keywords to search for.
keywords = [
    "heat wave", "heat action plan", "heat health", "heat early warning system",
    "heat-health action plan", "heat-health warning systems", 
    "extreme heat management", "heat wave management", "public health heat wave alert"
]

# Search Google and collect the URLs.
result_urls = search_google(keywords)

# Save the collected URLs to a CSV file.
with open("search_results_urls_22.csv", "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["URL"])  # Write the header
    
    for url in result_urls:
        writer.writerow([url])


##### b. Classify websites into categories-government, Journals, news sources, Non-govermental sites

In [6]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Function to classify websites based on keywords and URL patterns
def classify_websites(url):
    max_retries = 3
    retries = 0

    # Define keywords and URL patterns associated with different categories
    categories = {
        "Government": ["government", "official", "gov", "state", "federal", "administration", "public service"],
        "Academic Journal": ["journal", "research", "academia", "scholar", "academic", "study"],
        "News Source": ["news", "media", "press", "report", "headline", "journalism"],
        "Non Governmental Organization":["org","non-profit", "International"]
    }

    while retries < max_retries:
        try:
            # Get the webpage content (with SSL verification disabled)
            response = requests.get(url, verify=False)

            if response.status_code == 200:
                soup = BeautifulSoup(response.content, "html.parser")
                text = soup.get_text().lower()  # Convert to lowercase for case-insensitive matching

                # Classify the website based on keywords
                for category, keywords in categories.items():
                    for keyword in keywords:
                        if keyword in text:
                            return category

                # Classify the website based on URL patterns
                for category, url_patterns in categories.items():
                    for url_pattern in url_patterns:
                        if url_pattern in url.lower():
                            return category

            retries += 1
            time.sleep(2)  # Add a delay between retries (adjust as needed)

        except Exception as e:
            print(f"Attempt {retries + 1} failed for URL {url}. Error: {e}")
            print(f"Max retries reached for URL {url}. Classifying as 'Other'.")
            retries += 1
            time.sleep(2)  # Add a delay between retries (adjust as needed)

    # If still unable to classify using keywords and URL patterns, try URL-based classification
    for category, url_patterns in categories.items():
        for url_pattern in url_patterns:
            if url_pattern in url.lower():
                return category

    # If all methods fail, mark as 'other'
    return "Other"

# Load URLs from the input CSV file
input_csv = "search_results_urls_22.csv"
with open(input_csv, "r", encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile)
    urls = [row[0] for row in reader]

# Classify websites and store the results in a new CSV file
output_csv = "classified_websites_22.csv"
with open(output_csv, "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = ["URL", "Category"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for url in urls:
        category = classify_websites(url)
        writer.writerow({"URL": url, "Category": category})

print("Classification completed. Results are stored in 'classified_websites_22.csv'.")


Attempt 1 failed for URL URL. Error: Invalid URL 'URL': No schema supplied. Perhaps you meant http://URL?
Max retries reached for URL URL. Classifying as 'Other'.
Attempt 2 failed for URL URL. Error: Invalid URL 'URL': No schema supplied. Perhaps you meant http://URL?
Max retries reached for URL URL. Classifying as 'Other'.
Attempt 3 failed for URL URL. Error: Invalid URL 'URL': No schema supplied. Perhaps you meant http://URL?
Max retries reached for URL URL. Classifying as 'Other'.


C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'upload.wikimedia.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'upload.wikimedia.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'upload.wikimedia.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ifrc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ifrc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ifrc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.p

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'internal.imd.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'internal.imd.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.weather.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3

Attempt 1 failed for URL https://ghhin.org/heat-action-plans-and-case-studies/&amp;sa=U&amp;ved=2ahUKEwilo6Ws0NSBAxWmppUCHTq7Dd0QFnoECAIQAg&amp;usg=AOvVaw1FQTnWgfgAot47oGyV2NHf". Error: HTTPSConnectionPool(host='ghhin.org', port=443): Max retries exceeded with url: /heat-action-plans-and-case-studies/&amp;sa=U&amp;ved=2ahUKEwilo6Ws0NSBAxWmppUCHTq7Dd0QFnoECAIQAg&amp;usg=AOvVaw1FQTnWgfgAot47oGyV2NHf%22 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000016B5D9343D0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Max retries reached for URL https://ghhin.org/heat-action-plans-and-case-studies/&amp;sa=U&amp;ved=2ahUKEwilo6Ws0NSBAxWmppUCHTq7Dd0QFnoECAIQAg&amp;usg=AOvVaw1FQTnWgfgAot47oGyV2NHf". Classifying as 'Other'.


...

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ready.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ready.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ready.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpoo

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.iipa.org.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.iipa.org.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.iipa.org.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connect

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ndma.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ndma.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vikaspedia.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.health.vic.gov.au'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'acphd.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.gov.uk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpo

Classification completed. Results are stored in 'classified_websites_22.csv'.


### 3.0 Extract countries mentioned in the web pages extracted

##### 3.1 Google Scholar CSV

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import geonamescache

# Function to extract countries mentioned in webpage content
def extract_countries(url):
    # Get the webpage content (with SSL verification disabled)
    response = requests.get(url, verify=False)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        text = soup.get_text().lower()  # Convert to lowercase for case-insensitive matching

        # Create a list to store countries mentioned in the content
        mentioned_countries = []

        # Load country names using geonamescache
        gc = geonamescache.GeonamesCache()
        countries = gc.get_countries_by_names()

        # Check if country names are mentioned in the content
        for country_name in countries:
            if country_name.lower() in text:
                mentioned_countries.append(country_name)

        return mentioned_countries

    return []  # Return an empty list if there's an issue with the webpage

# Load classified websites from the input CSV file
input_csv = "google_scholar_results_with_abstracts_fixed_2.csv"  
output_csv = "google_scholar_results_with_countries_2.csv"  

with open(input_csv, "r", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    rows = list(reader)

# Add a new 'Countries' column to the CSV
fieldnames = reader.fieldnames + ["Countries"]

with open(output_csv, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for row in rows:
        url = row["Link"]
        mentioned_countries = extract_countries(url)

        # Update the row with the list of mentioned countries
        row["Countries"] = ", ".join(mentioned_countries)

        # Write the updated row to the output CSV
        writer.writerow(row)

print(f"Country extraction completed. Results are stored in '{output_csv}'.")


C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.sciencedirect.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.sciencedirect.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nature.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urll

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tandfonline.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'journals.aps.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ehp.niehs.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nature.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'agupubs.onlinelibrary.wiley.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ajph.aphapublications.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'link.springer.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'idp.springer.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'link.springer.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\co

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.mdpi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'link.springer.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'idp.springer.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connect

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.sciencedirect.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.worldscientific.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.annualreviews.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-pack

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'academic.oup.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'academic.oup.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Country extraction completed. Results are stored in 'google_scholar_results_with_countries_2.csv'.


##### 3.2 Check for presence of key terms in the webpage content for google scholar and append columns for each key term. Then assign bolean values 0 or 1

In [2]:
import requests
from bs4 import BeautifulSoup
import csv

# List of keywords to search for.
keywords = ["heatwave", "heat wave", "heat  wave", "heat  action  plan", "heat  health", "heat early warning system",
            "heat-health action plan", "heat-health warning  systems", "extreme heat management", "heat wave management",
            "public health heat wave alert"]

# Define the input and output CSV file paths
input_csv = "google_scholar_results_with_countries_2.csv"
output_csv = "google_scholar_results_with_keyterms_booleans_2.csv"

# Function to extract text content from a webpage
def extract_webpage_content(url):
    try:
        response = requests.get(url, verify=False)
        soup = BeautifulSoup(response.content, "html.parser")
        return soup.get_text().lower()  # Convert to lowercase for case-insensitive matching
    except Exception as e:
        return ""

# Scrape Google Scholar for each keyword and update the existing CSV file
with open(input_csv, "r", encoding="utf-8") as input_file, open(output_csv, "w", newline="", encoding="utf-8") as output_file:
    # Create a CSV reader and writer
    reader = csv.DictReader(input_file)
    fieldnames = reader.fieldnames + keywords  # Add columns for key terms
    writer = csv.DictWriter(output_file, fieldnames=fieldnames)
    
    # Write the header to the output CSV
    writer.writeheader()

    # Iterate over each row in the input CSV
    for row in reader:
        # Scrape the content of the webpage linked in the row
        url = row["Link"]
        webpage_content = extract_webpage_content(url)

        # Initialize a dictionary for key term presence (1 or 0)
        key_term_presence = {keyword: 1 if keyword.lower() in webpage_content else 0 for keyword in keywords}

        # Update the row data with key term presence
        row.update(key_term_presence)

        # Write the updated row to the output CSV
        writer.writerow(row)

print(f"Data saved to '{output_csv}'.")


C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.sciencedirect.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.sciencedirect.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nature.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urll

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tandfonline.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'journals.aps.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ehp.niehs.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nature.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'agupubs.onlinelibrary.wiley.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ajph.aphapublications.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'link.springer.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'idp.springer.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'link.springer.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\co

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.mdpi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'link.springer.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'idp.springer.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connect

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.sciencedirect.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.worldscientific.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.annualreviews.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-pack

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'academic.oup.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'academic.oup.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Data saved to 'google_scholar_results_with_keyterms_booleans_2.csv'.


#### 3.3 Check countries mention from google search results csv

In [4]:
import csv
import requests
from bs4 import BeautifulSoup
from geonamescache import GeonamesCache

# Define the input and output CSV file paths
input_csv = "classified_websites_22.csv"  # Replace with the path to your input CSV file
output_csv = "Google_search_result_with_mentioned_places.csv"

# Initialize GeonamesCache
gc = GeonamesCache()

# Get a list of country names and city names
countries = [country["name"] for country in gc.get_countries().values()]
#cities = [city["name"] for city in gc.get_cities().values()]

# Function to extract mentioned places from text
def extract_mentioned_places(text):
    mentioned_places = []
    for place in countries:
        if place.lower() in text.lower():
            mentioned_places.append(place)
    if mentioned_places:
        return ', '.join(mentioned_places)
    else:
        return "None"

# Read URLs from the input CSV and add a column for mentioned places
with open(input_csv, "r", encoding="utf-8") as input_file, open(output_csv, "w", newline="", encoding="utf-8") as output_file:
    # Create a CSV reader and writer
    reader = csv.DictReader(input_file)
    fieldnames = reader.fieldnames + ["Mentioned Places"]  # Add a column for mentioned places
    writer = csv.DictWriter(output_file, fieldnames=fieldnames)
    
    # Write the header to the output CSV
    writer.writeheader()

    # Iterate over each row in the input CSV
    for row in reader:
        url = row["URL"]
        
        try:
            # Get the webpage content
            response = requests.get(url, verify=False)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, "html.parser")
                text = soup.get_text()
                
                # Extract mentioned places (countries and cities)
                mentioned_places = extract_mentioned_places(text)
                
                # Update the row data with mentioned places
                row["Mentioned Places"] = mentioned_places
        except Exception as e:
            print(f"Error processing URL {url}: {e}")
        
        # Write the updated row to the output CSV
        writer.writerow(row)

print(f"Data saved to '{output_csv}'.")


Error processing URL URL: Invalid URL 'URL': No schema supplied. Perhaps you meant http://URL?


C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'upload.wikimedia.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'd2jx2rerrg6sh3.cloudfront.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'thumbs.dreamstime.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.redcross.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'heatactionplatform.onebillionresilient.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'heatactionplatform.onebillionresilient.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\P

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.dir.ca.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cdc.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cdc.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.p

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'my.clevelandclinic.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'my.clevelandclinic.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'my.clevelandclinic.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-pac

...

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'climate-adapt.eea.europa.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.who.int'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.who.int'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\con

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.safework.nsw.gov.au'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.washington.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.washington.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\u

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.preventionweb.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartnet.niua.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ndma.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\con

Data saved to 'Google_search_result_with_mentioned_places.csv'.


##### 3.4 Check for the key terms in the google search results, append columns for each and assign bolean 1 for presence otherwise 0

In [5]:
import csv
import requests
from bs4 import BeautifulSoup

# Define the input and output CSV file paths
input_csv = "Google_search_result_with_mentioned_places.csv"  # Replace with the path to your input CSV file
output_csv = "Google_search_result_with_key_terms.csv"

# List of keywords to search for
keywords = ["heatwave", "heat wave", "heat  wave", "heat  action  plan", "heat  health", "heat early warning system",
            "heat-health action plan", "heat-health warning  systems", "extreme heat management", "heat wave management",
            "public health heat wave alert"]

# Function to check for the presence of key terms in text
def check_key_terms(text):
    presence = {keyword: 1 if keyword.lower() in text.lower() else 0 for keyword in keywords}
    return presence

# Read URLs from the input CSV and add columns for key terms
with open(input_csv, "r", encoding="utf-8") as input_file, open(output_csv, "w", newline="", encoding="utf-8") as output_file:
    # Create a CSV reader and writer
    reader = csv.DictReader(input_file)
    fieldnames = reader.fieldnames + keywords  # Add columns for key terms
    writer = csv.DictWriter(output_file, fieldnames=fieldnames)
    
    # Write the header to the output CSV
    writer.writeheader()

    # Iterate over each row in the input CSV
    for row in reader:
        url = row["URL"]
        
        try:
            # Get the webpage content
            response = requests.get(url, verify=False)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, "html.parser")
                text = soup.get_text()
                
                # Check for the presence of key terms
                key_term_presence = check_key_terms(text)
                
                # Update the row data with key term presence
                row.update(key_term_presence)
        except Exception as e:
            print(f"Error processing URL {url}: {e}")
        
        # Write the updated row to the output CSV
        writer.writerow(row)

print(f"Data saved to '{output_csv}'.")


Error processing URL URL: Invalid URL 'URL': No schema supplied. Perhaps you meant http://URL?


C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'upload.wikimedia.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'd2jx2rerrg6sh3.cloudfront.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'thumbs.dreamstime.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.vox.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.redcross.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'heatactionplatform.onebillionresilient.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-p

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.betterhealth.vic.gov.au'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.dir.ca.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cdc.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'my.clevelandclinic.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'my.clevelandclinic.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'my.clevelandclinic.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-pac

...

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'climate-adapt.eea.europa.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.who.int'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.who.int'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\con

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.safework.nsw.gov.au'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.washington.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.washington.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\u

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.preventionweb.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartnet.niua.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ndma.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\con

Data saved to 'Google_search_result_with_key_terms.csv'.


In [1]:
import pandas as pd

pd.read_csv('Google_search_result_Final.csv')

,URL,Category,Mentioned Places,heatwave,heat wave,heat wave,heat action plan,heat health,heat early warning system,heat-health action plan,heat-health warning systems,extreme heat management,heat wave management,public health heat wave alert
0,URL,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://upload.wikimedia.org/wikipedia/commons...,News Source,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://d2jx2rerrg6sh3.cloudfront.net/images/A...,News Source,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://thumbs.dreamstime.com/b/heat-wave-conc...,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://cdn.abcotvs.com/dip/images/5343664_061...,Other,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,http://www.googleadservices.com/')})();window....,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159,https://www.cdc.gov/climateandhealth/pubs/extr...,Government,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,https://www.weather.gov/safety/heat-ww\x26amp;...,Government,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161,https://www.uwinnipeg.ca/emergency-guidelines/...,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Load your DataFrame
df = pd.read_csv('Google_search_result_Final.csv')

# Function to fetch the title of a webpage given a URL
def get_title(url):
    try:
        # Send an HTTP GET request to the URL with SSL/TLS verification enabled
        response = requests.get(url, verify=True)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content of the page
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find the title tag in the HTML
            title_tag = soup.find('title')

            # Extract and return the title text
            if title_tag:
                return title_tag.text.strip()
            else:
                return None
        else:
            return None
    except Exception as e:
        print(f"Error fetching title for {url}: {e}")
        return None

# Create a new column 'titles' and fill it with the titles of the webpages
df['titles'] = df['URL'].apply(get_title)

# Save the updated DataFrame to a new CSV file
df.to_csv('Google_search_result_Final_with_titles.csv', index=False)


Error fetching title for URL: Invalid URL 'URL': No schema supplied. Perhaps you meant http://URL?
Error fetching title for https://my.clevelandclinic.org/health/diseases/16425-heat-illness\x26amp;sa\x3dU\x26amp;ved\x3d2ahUKEwjxppWt0NSBAxX3p5UCHY8ZAi4QFnoECAIQFg\x26amp;usg\x3dAOvVaw0KDeMmK7y4gNQnlTZ83saL\x22: Exceeded 30 redirects.
Error fetching title for https://community.fema.gov/ProtectiveActions/s/article/Extreme-Heat&amp;sa=U&amp;ved=2ahUKEwiXiKSw0NSBAxWhp5UCHQwRCmUQFnoECAgQAg&amp;usg=AOvVaw33aTjUggkzkGiuNBQpolxN": HTTPSConnectionPool(host='community.fema.gov', port=443): Max retries exceeded with url: /ProtectiveActions/s/article/Extreme-Heat&amp;sa=U&amp;ved=2ahUKEwiXiKSw0NSBAxWhp5UCHQwRCmUQFnoECAgQAg&amp;usg=AOvVaw33aTjUggkzkGiuNBQpolxN%22 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)')))
Error fetching title for https://vikaspedia.in/social-welfare/disaster-mana